In [1]:
import os
import torch
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/uj-user/Yo/hybrid-ltm/ltm-venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test.jsonl"
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test_last_session.jsonl"
# data_path = "/home/nayo/HumanEval/context-fms/MS_DG/test.jsonl"
model_name = "microsoft/DialoGPT-large"
# model_name = "gpt2"
MODEL_MAX_LEN=1024
TARGET_LEN = 128
device = "cuda:3" if torch.cuda.is_available() else "cpu"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [4]:
dataset = pd.read_json(data_path, lines=True)
dataset

,input
0,```\nDialogue Session #3:\nDoctor:Hello Patien...
1,```\nDialogue Session #3:\nNeighbors A:Hey the...
2,"```\nDialogue Session #4:\nCo-workers A:Hey, B..."
3,"```\nDialogue Session #2:\nMentee:Mentor, I ha..."
4,```\nDialogue Session #5:\nClassmates A:Sorry ...
...,...
75,```\nDialogue Session #4:\nHusband:You know wh...
76,"```\nDialogue Session #4:\nClassmates B:Hey, C..."
77,```\nDialogue Session #3:\nClassmates A:I've b...
78,```\nDialogue Session #2:\nHusband:I found a s...


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left', use_fast=False)
# tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'})
tokenizer

GPT2Tokenizer(name_or_path='microsoft/DialoGPT-large', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [6]:
import re
dataset['prep_input'] = dataset['input'].map(lambda x: '```'+ '```'.join(x.split('```')[1:-1]) +'```')
text_list = dataset['prep_input'][1].split('Dialogue Session #')[1:] # dialogue session remove
result_list = [re.sub(r'^\d+:\s*', '', line) for line in text_list] # #1 숫자 제거
result_list = ['\n'.join(line.split(':', 1)[1].strip() for line in lines.split('\n') if ':' in line) for lines in result_list]
result_list = [line.replace('\n', tokenizer.eos_token).replace('```', '').strip() for line in result_list] # 불필요 개행문자, 공백제거
result_list = ' '.join(result_list)
result_list = result_list[:-3].strip()
result_list

'Hey there, have you ever considered pursuing journalism as a career?<|endoftext|>Journalism? No, not really. Why do you ask?<|endoftext|>'

In [7]:
ans_jsons = []
for i, line in enumerate(tqdm(dataset['input'])):
    dataset['prep_input'] = dataset['input'].map(lambda x: '```'+ '```'.join(x.split('```')[1:-1]) +'```')
    text_list = dataset['prep_input'][i].split('Dialogue Session #')[1:]
    result_list = [re.sub(r'^\d+:\s*', '', line) for line in text_list] # #1 숫자 제거
    result_list = ['\n'.join(line.split(':', 1)[1].strip() for line in lines.split('\n') if ':' in line) for lines in result_list]
    result_list = [line.replace('\n', tokenizer.eos_token).replace('```', '').strip() for line in result_list] # 불필요 개행문자, 공백제거
    result_list = ' '.join(result_list)
    result_list = result_list[:-3].strip()
    # print(result_list)

    input_ids = tokenizer.encode(result_list, max_length=100, truncation=True, return_tensors='pt').to(device)#.input_ids
    # print(input_ids)
    target_len = min(len(input_ids[0]) + TARGET_LEN, MODEL_MAX_LEN)
    output_ids = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.2,
        max_length=MODEL_MAX_LEN,
        pad_token_id=tokenizer.eos_token_id,
    )

    output_ids = output_ids[0][len(input_ids[0]):]
    # print(input_ids[0].shape)
    # print(output_ids.shape)
    outputs = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    # print('input:\n', tokenizer.decode(input.input_ids[0]))
    # print('output:\n', outputs)
    # if i%10==8:
    #     break
    
    ans_jsons.append(
        {
            "question_id": i,
            "text": outputs,
            "model_id": model_name,
            "metadata": {},
        }
    )

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


torch.Size([5])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


torch.Size([11])
torch.Size([6])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

torch.Size([7])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


torch.Size([9])
torch.Size([8])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


torch.Size([16])
torch.Size([5])
torch.Size([20])


In [8]:
filename = f"predictions/{model_name}_{data_path.split('/')[-2]}.json"
save_path = os.path.dirname(filename)
os.makedirs(save_path, exist_ok=True)
df_ans_json = pd.DataFrame(ans_jsons)
df_ans_json.to_json(filename, orient='records', lines=True)

In [9]:
for step in range(5):
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt').to(device)
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    bot_input_ids = bot_input_ids.to(device)
    # print('bot_input_ids:', bot_input_ids.shape)

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:][0], skip_special_tokens=False)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: <|endoftext|><|endoftext|>


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: <|endoftext|><|endoftext|><|endoftext|><|endoftext|>


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [10]:
print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:][0], skip_special_tokens=False)))

DialoGPT: <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
